# 4-1: Find QARTOD tests not executed 
This notebook searches through datasets for CGSN instruments and QARTOD automated test lookup tables for tests in production that are not applied to data streams as expected.

In [1]:
# Import libraries
import numpy as np
import pandas as pd
import xarray as xr
import requests
import io
import ast
from glob import glob

In [2]:
# Import functions from ooinet and ooi_data_explorations libraries
from ooi_data_explorations.common import load_kdata, get_vocabulary, m2m_request, m2m_collect
from ooinet import M2M

In [3]:
# Import functions from project qc_completion module
from qartod_testing.qc_completion import load_gross_range_qartod_test_list, \
    load_climatology_qartod_test_list, make_test_parameter_dict, \
    check_tests_exe, make_results_table, add_test_exe, write_results

In [4]:
# Define site for refdes search and find datasets available
site = 'CP04OSPM'
datasets = M2M.search_datasets(site)
datasets.reset_index(inplace=True)
datasets.drop(labels="index", axis=1, inplace=True)
datasets[0:5]

Searching https://ooinet.oceanobservatories.org/api/m2m/12576/sensor/inv/CP04OSPM


,array,node,instrument,refdes,url,deployments
0,CP04OSPM,WFP01,05-PARADK000,CP04OSPM-WFP01-05-PARADK000,https://ooinet.oceanobservatories.org/api/m2m/...,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
1,CP04OSPM,WFP01,04-FLORTK000,CP04OSPM-WFP01-04-FLORTK000,https://ooinet.oceanobservatories.org/api/m2m/...,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
2,CP04OSPM,WFP01,03-CTDPFK000,CP04OSPM-WFP01-03-CTDPFK000,https://ooinet.oceanobservatories.org/api/m2m/...,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
3,CP04OSPM,WFP01,02-DOFSTK000,CP04OSPM-WFP01-02-DOFSTK000,https://ooinet.oceanobservatories.org/api/m2m/...,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
4,CP04OSPM,WFP01,01-VEL3DK000,CP04OSPM-WFP01-01-VEL3DK000,https://ooinet.oceanobservatories.org/api/m2m/...,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."


### Run cross-reference for tests executed on all refdes and datastreams at site

The cell as written below ran for all refdes-stream combinations printed below the cell until the kernel crashed. At this point, this cell and the cells above it were turned into a python script `run_qartod_test_cross-ref.py`.

In [ ]:
# Set csv save directory and file name for results
csv_name = "GA01SUMO_test_cross-ref_results.csv"
csv_dir = "./../data/processed/"
# loop through sensors to check and find datastreams available
for k in datasets.index:
    refdes = datasets.refdes[k]
    datastreams = M2M.get_datastreams(refdes)
    # loop through datastreams and first deployment available
    site, node, sensor = refdes.split("-", 2)
    for m in datastreams.index:
        method = datastreams.method[m]
        stream = datastreams.stream[m]
        deploy = datasets.deployments[k][0]
        instclass = sensor[3:8]
        # Load gross range and climatology test tables
        grt_table = load_gross_range_qartod_test_list(refdes, stream)
        ct_table = load_climatology_qartod_test_list(refdes, stream)
        if (grt_table is False) and (ct_table is False):
            print(f"No test in production for {refdes}-{stream}.")
        else:
            # Load data
            get_vocabulary(site, node, sensor)
            data = load_kdata(site, node, sensor, method, stream, ('*deployment%04d*%s*.nc' % (deploy, instclass)))
            try:
                print(data.id) # to check data stream loaded
            except AttributeError:
                print(f"No dataset available for {refdes}-{stream} tests.")
                del [grt_table, ct_table]
            else:
                # Create a dictionary of key-value pairs of dataset variable name:alternate parameter name
                test_parameters = make_test_parameter_dict(data)
                # Use test parameters to check for tests executed in dataset
                test_exe = check_tests_exe(data, test_parameters, grt_table, ct_table)
                # Make table for cross-ref results
                table = make_results_table(grt_table, ct_table)
                # Add column with QARTOD tests executed by parameter
                table = add_test_exe(table, test_exe)
                # Write QARTOD test cross-reference results table to a CSV
                write_results(table, csv_name, csv_dir)
                del [grt_table, ct_table, data, test_parameters,
                     test_exe, table]

### Run cross-reference for tests executed for a single datastream

In [5]:
# Pick a sensor to check and find datastreams available
n = 4
refdes = datasets.refdes[n]
datastreams = M2M.get_datastreams(refdes)

In [6]:
datastreams

,refdes,method,stream
0,CP04OSPM-WFP01-01-VEL3DK000,recovered_wfp,vel3d_k_wfp_instrument
1,CP04OSPM-WFP01-01-VEL3DK000,recovered_wfp,vel3d_k_wfp_metadata
2,CP04OSPM-WFP01-01-VEL3DK000,telemetered,vel3d_k_wfp_stc_instrument
3,CP04OSPM-WFP01-01-VEL3DK000,telemetered,vel3d_k_wfp_stc_metadata


In [7]:
# Select first datastream and first deployment available
m = 2
site, node, sensor = refdes.split("-", 2)
method = datastreams.method[m]
stream = datastreams.stream[m]
deploy = datasets.deployments[n][0]
instclass = sensor[3:8]
print([site, node, sensor, method, stream, deploy, instclass])

['CP04OSPM', 'WFP01', '01-VEL3DK000', 'telemetered', 'vel3d_k_wfp_stc_instrument', 1, 'VEL3D']


In [8]:
# Load data
get_vocabulary(site, node, sensor)
print(f"Loading deployment {deploy}")
data = load_kdata(site, node, sensor, method, stream,
                  ('*deployment%04d*%s*.nc' % (deploy, instclass)))
while data is None:
    deploy+=1
    if deploy > datasets.deployments[n][-1]:
        print("No dataset loaded")
        break
    else:
        print(f"Loading deployment {deploy}")
        data = load_kdata(site, node, sensor, method, stream,
                          ('*deployment%04d*%s*.nc' % (deploy, instclass)))

Loading deployment 1


Loading and Processing Data Files: 0it [00:00, ?it/s]
/home/jovyan/code/ooi-data-explorations/python/ooi_data_explorations/common.py:855: UserWarning: No data files were loaded from the JupyterHub kdata directory.
  warnings.warn(message)


Loading deployment 2


Loading and Processing Data Files: 100%|██████████| 1/1 [00:02<00:00,  2.06s/it]


In [9]:
# Check dataset contents
data

<xarray.Dataset>
Dimensions:                                 (time: 2095836,
                                             date_time_array_dim_0: 6,
                                             vel3d_k_data_set_description_dim_0: 5)
Coordinates:
  * date_time_array_dim_0                   (date_time_array_dim_0) int32 0 ....
  * time                                    (time) datetime64[ns] 2014-10-04T...
  * vel3d_k_data_set_description_dim_0      (vel3d_k_data_set_description_dim_0) int32 ...
Data variables: (12/29)
    vel3d_k_vel1                            (time) int16 240 -46 ... 253 121
    vel3d_k_northward_velocity_qc_results   (time) uint8 29 29 29 ... 29 29 29
    lon                                     (time) float64 -70.88 ... -70.88
    lat                                     (time) float64 39.94 39.94 ... 39.94
    vel3d_k_heading                         (time) uint16 680 680 ... 1893 1893
    vel3d_k_temp_c                          (time) int16 2318 2318 ... 1102 1102
    ...                                      ...
    vel3d_k_pitch                           (time) int16 -27 -27 -17 ... -17 -17
    vel3d_k_data_set_description            (time, vel3d_k_data_set_description_dim_0) uint16 ...
    int_ctd_pressure                        (time) float64 30.61 30.61 ... 57.2
    depth                                   (time) float64 30.37 30.37 ... 56.76
    vel3d_k_roll                            (time) int16 -56 -56 30 ... -2 0 0
    vel3d_k_eastward_velocity_qc_executed   (time) uint8 29 29 29 ... 29 29 29
Attributes: (12/67)
    node:                               WFP01
    comment:                            Data produced by the OOI M2M API and ...
    publisher_email:                    
    sourceUrl:                          http://oceanobservatories.org/
    collection_method:                  telemetered
    stream:                             vel3d_k_wfp_stc_instrument
    ...                                 ...
    geospatial_lon_max:                 -70.87843
    geospatial_lon_units:               degrees_east
    geospatial_lon_resolution:          0.1
    geospatial_vertical_units:          meters
    geospatial_vertical_resolution:     0.1
    geospatial_vertical_positive:       down

In [11]:
# Load gross range and climatology test tables
grt_table = load_gross_range_qartod_test_list(refdes, stream)
ct_table = load_climatology_qartod_test_list(refdes, stream)

In [12]:
# Create a dictionary of key-value pairs of dataset variable name:alternate parameter name
test_parameters = make_test_parameter_dict(data)
test_parameters

{}

In [13]:
# Loop through table parameters to check for tests executed
test_exe = check_tests_exe(data, test_parameters, grt_table, ct_table)
test_exe

{'vel3d_k_temp_c': 'none',
 'vel3d_k_eastward_velocity': 'none',
 'vel3d_k_northward_velocity': 'none',
 'vel3d_k_upward_velocity': 'none',
 'int_ctd_pressure': 'none',
 'vel3d_k_pitch': 'none',
 'vel3d_k_roll': 'none'}

In [14]:
grt_table

,subsite,node,sensor,stream,parameters
161,CP04OSPM,WFP01,01-VEL3DK000,vel3d_k_wfp_stc_instrument,vel3d_k_temp_c
162,CP04OSPM,WFP01,01-VEL3DK000,vel3d_k_wfp_stc_instrument,vel3d_k_eastward_velocity
163,CP04OSPM,WFP01,01-VEL3DK000,vel3d_k_wfp_stc_instrument,vel3d_k_northward_velocity
164,CP04OSPM,WFP01,01-VEL3DK000,vel3d_k_wfp_stc_instrument,vel3d_k_upward_velocity
165,CP04OSPM,WFP01,01-VEL3DK000,vel3d_k_wfp_stc_instrument,int_ctd_pressure
166,CP04OSPM,WFP01,01-VEL3DK000,vel3d_k_wfp_stc_instrument,vel3d_k_pitch
167,CP04OSPM,WFP01,01-VEL3DK000,vel3d_k_wfp_stc_instrument,vel3d_k_roll


In [15]:
ct_table

False

In [16]:
table = make_results_table(grt_table, ct_table)
table

,subsite,node,sensor,stream,parameters,GRTtable,CTtable
0,CP04OSPM,WFP01,01-VEL3DK000,vel3d_k_wfp_stc_instrument,vel3d_k_temp_c,True,False
1,CP04OSPM,WFP01,01-VEL3DK000,vel3d_k_wfp_stc_instrument,vel3d_k_eastward_velocity,True,False
2,CP04OSPM,WFP01,01-VEL3DK000,vel3d_k_wfp_stc_instrument,vel3d_k_northward_velocity,True,False
3,CP04OSPM,WFP01,01-VEL3DK000,vel3d_k_wfp_stc_instrument,vel3d_k_upward_velocity,True,False
4,CP04OSPM,WFP01,01-VEL3DK000,vel3d_k_wfp_stc_instrument,int_ctd_pressure,True,False
5,CP04OSPM,WFP01,01-VEL3DK000,vel3d_k_wfp_stc_instrument,vel3d_k_pitch,True,False
6,CP04OSPM,WFP01,01-VEL3DK000,vel3d_k_wfp_stc_instrument,vel3d_k_roll,True,False


In [17]:
# Add column with QARTOD tests executed by parameter
table = add_test_exe(table, test_exe)
table

,subsite,node,sensor,stream,parameters,GRTtable,CTtable,testsExecuted
0,CP04OSPM,WFP01,01-VEL3DK000,vel3d_k_wfp_stc_instrument,vel3d_k_temp_c,True,False,none
1,CP04OSPM,WFP01,01-VEL3DK000,vel3d_k_wfp_stc_instrument,vel3d_k_eastward_velocity,True,False,none
2,CP04OSPM,WFP01,01-VEL3DK000,vel3d_k_wfp_stc_instrument,vel3d_k_northward_velocity,True,False,none
3,CP04OSPM,WFP01,01-VEL3DK000,vel3d_k_wfp_stc_instrument,vel3d_k_upward_velocity,True,False,none
4,CP04OSPM,WFP01,01-VEL3DK000,vel3d_k_wfp_stc_instrument,int_ctd_pressure,True,False,none
5,CP04OSPM,WFP01,01-VEL3DK000,vel3d_k_wfp_stc_instrument,vel3d_k_pitch,True,False,none
6,CP04OSPM,WFP01,01-VEL3DK000,vel3d_k_wfp_stc_instrument,vel3d_k_roll,True,False,none


In [97]:
# Write QARTOD test cross-reference results table to a CSV 
csv_name = "test_cross-ref_results.csv"
csv_dir = "./../data/processed/"
write_results(table, csv_name, csv_dir)

results saved to ./../data/processed/test_cross-ref_results.csv


In [43]:
# Load data
get_vocabulary('CE09OSPM', 'WFP01', '01-VEL3DK000')
data = load_kdata('CE09OSPM', 'WFP01', '01-VEL3DK000', 'recovered_wfp', 'vel3d_k_wfp_instrument', ('*deployment%04d*%s*.nc' % (2, "VEL3D")))

Loading and Processing Data Files: 100%|██████████| 1/1 [00:06<00:00,  6.52s/it]


In [44]:
data

<xarray.Dataset>
Dimensions:                                 (time: 4546642,
                                             date_time_array_dim_0: 6,
                                             vel3d_k_data_set_description_dim_0: 5)
Coordinates:
  * date_time_array_dim_0                   (date_time_array_dim_0) int32 0 ....
  * time                                    (time) datetime64[ns] 2014-10-08T...
  * vel3d_k_data_set_description_dim_0      (vel3d_k_data_set_description_dim_0) int32 ...
Data variables: (12/56)
    vel3d_k_version                         (time) uint8 2 2 2 2 2 ... 2 2 2 2 2
    vel3d_k_id                              (time) uint8 21 21 21 ... 21 21 21
    vel3d_k_vel1                            (time) int16 160 233 93 ... 129 110
    vel3d_k_error                           (time) uint16 0 0 0 0 0 ... 0 0 0 0
    vel3d_k_pressure_qc_results             (time) uint8 1 1 1 1 1 ... 1 1 1 1 1
    lon                                     (time) float64 -125.0 ... -125.0
    ...                                      ...
    depth                                   (time) float64 41.99 41.99 ... 506.5
    vel3d_k_temp_c                          (time) int16 864 864 862 ... 549 549
    vel3d_k_ambiguity                       (time) int16 10714 10714 ... 10714
    vel3d_k_roll                            (time) int16 15 15 13 13 ... 0 0 0 0
    vel3d_k_amp0                            (time) uint8 117 101 110 ... 97 93
    vel3d_k_eastward_velocity_qc_executed   (time) uint8 29 29 29 ... 29 29 29
Attributes: (12/67)
    node:                               WFP01
    comment:                            Data produced by the OOI M2M API and ...
    publisher_email:                    
    sourceUrl:                          http://oceanobservatories.org/
    collection_method:                  recovered_wfp
    stream:                             vel3d_k_wfp_instrument
    ...                                 ...
    geospatial_lon_max:                 -124.9663
    geospatial_lon_units:               degrees_east
    geospatial_lon_resolution:          0.1
    geospatial_vertical_units:          meters
    geospatial_vertical_resolution:     0.1
    geospatial_vertical_positive:       down

In [10]:
m2m_result = m2m_request('GP02HYPM', 'WFP03', '05-VEL3DL000', 'recovered_wfp', 'vel3d_l_wfp_instrument_recovered',
                         start="2021-08-01T00:00:00.000Z", stop="2021-10-01T00:00:00.000Z")

Requesting:
	refdes: GP02HYPM-WFP03-05-VEL3DL000
	method: recovered_wfp
	stream: vel3d_l_wfp_instrument_recovered
	from 2021-08-01T00:00:00.000Z to 2021-10-01T00:00:00.000Z
Waiting for OOINet to process and prepare data request, this may take up to 20 minutes.
Waiting: 100%|██████████| 400/400 [01:30<00:00,  4.44it/s]


In [11]:
m2m_data = m2m_collect(m2m_result)

In [12]:
m2m_data

<xarray.Dataset>
Dimensions:                                             (time: 1095300,
                                                         vel3d_l_date_time_array_dim_0: 6)
Coordinates:
  * vel3d_l_date_time_array_dim_0                       (vel3d_l_date_time_array_dim_0) int32 ...
  * time                                                (time) datetime64[ns] ...
Data variables: (12/42)
    vel3d_l_hz                                          (time) float32 nan .....
    vel3d_l_hx                                          (time) float32 nan .....
    vel3d_l_hy                                          (time) float32 nan .....
    vel3d_l_upward_velocity_ascending_qartod_executed   (time) object nan ......
    vel3d_l_eastward_velocity                           (time) float64 nan .....
    lon                                                 (time) float64 -144.8...
    ...                                                  ...
    depth                                               (time) float64 2.13e+...
    vel3d_l_vp1                                         (time) float32 nan .....
    vel3d_l_vp3                                         (time) float32 nan .....
    vel3d_l_tx_qartod_executed                          (time) object nan ......
    vel3d_l_upward_velocity_ascending                   (time) float32 nan .....
    sea_water_pressure                                  (time) float64 2.159e...
Attributes: (12/67)
    node:                               WFP03
    comment:                            Data produced by the OOI M2M API and ...
    publisher_email:                    
    sourceUrl:                          http://oceanobservatories.org/
    collection_method:                  recovered_wfp
    stream:                             vel3d_l_wfp_instrument_recovered
    ...                                 ...
    geospatial_lon_max:                 -144.800675
    geospatial_lon_units:               degrees_east
    geospatial_lon_resolution:          0.1
    geospatial_vertical_units:          meters
    geospatial_vertical_resolution:     0.1
    geospatial_vertical_positive:       down

In [14]:
deploy_info = M2M.get_deployments(refdes, deploy_num='3')

In [24]:
t = deploy_info.deployStart+pd.Timedelta(5, 'D')

In [29]:
t[0].isoformat(timespec='milliseconds')+'Z'

'2015-05-04T20:15:00.000Z'